In [ ]:
# !pip install llama-index llama-index-vector-stores-mongodb llama-index-embeddings-openai pymongo

In [ ]:
import getpass, os, pymongo, pprint
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext
from llama_index.core.settings import Settings
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.vector_stores import MetadataFilter, MetadataFilters, ExactMatchFilter, FilterOperator
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.vector_stores.mongodb import MongoDBAtlasVectorSearch

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_KEY")
ATLAS_CONNECTION_STRING = os.environ.get("ATLAS_URI")

In [ ]:
Settings.llm = OpenAI()
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")
Settings.chunk_size = 100
Settings.chunk_overlap = 10

In [ ]:
# import requests
# import os

# # Define the URL and the file name
# url = 'https://query.prod.cms.rt.microsoft.com/cms/api/am/binary/RE4HkJP'
# file_name = 'data/atlas_best_practices.pdf'

# # Create the 'data' directory if it doesn't exist
# os.makedirs(os.path.dirname(file_name), exist_ok=True)

# # Download the file
# response = requests.get(url)
# if response.status_code == 200:
#     with open(file_name, 'wb') as f:
#         f.write(response.content)
#     print("File downloaded successfully.")
# else:
#     print("Failed to download file. Status code:", response.status_code)

In [ ]:
# Load the sample data
sample_data = SimpleDirectoryReader(input_files=["./data/atlas_best_practices.pdf"]).load_data()

# Print the first document
sample_data[0]

In [ ]:
# Connect to your Atlas cluster
mongodb_client = pymongo.MongoClient(ATLAS_CONNECTION_STRING)

# Instantiate the vector store
atlas_vector_search = MongoDBAtlasVectorSearch(
    mongodb_client,
    db_name = "llamaindex_test_db",
    collection_name = "test",
    index_name = "vector_index"
)
vector_store_context = StorageContext.from_defaults(vector_store=atlas_vector_search)

In [ ]:
vector_store_index = VectorStoreIndex.from_documents(
   sample_data, storage_context=vector_store_context, show_progress=True
)

In [ ]:
#search index
# {
#   "mappings": {
#     "dynamic": true,
#     "fields": {
#       "embedding": {
#         "dimensions": 1536,
#         "similarity": "cosine",
#         "type": "knnVector"
#       }
#     }
#   }
# }

In [ ]:
# Instantiate Atlas Vector Search as a retriever
vector_store_retriever = VectorIndexRetriever(index=vector_store_index, similarity_top_k=5)
# Pass the retriever into the query engine
query_engine = RetrieverQueryEngine(retriever=vector_store_retriever)
# Prompt the LLM
response = query_engine.query('How can I secure my MongoDB Atlas cluster?')

In [ ]:
print(response)
print("\nSource documents: ")
pprint.pprint(response.source_nodes)

## Eval With TrueLens

In [ ]:
from trulens_eval import Tru
tru = Tru()

In [ ]:
import numpy as np

# Initialize provider class
from trulens_eval.feedback.provider.openai import OpenAI
openai = OpenAI()

# select context to be used in feedback. the location of context is app specific.
from trulens_eval.app import App
context = App.select_context(query_engine)

# imports for feedback
from trulens_eval import Feedback

# Define a groundedness feedback function
from trulens_eval.feedback import Groundedness
grounded = Groundedness(groundedness_provider=OpenAI())
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons)
    .on(context.collect()) # collect context chunks into a list
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(openai.relevance).on_input_output()

# Question/statement relevance between question and each context chunk.
f_qs_relevance = (
    Feedback(openai.qs_relevance)
    .on_input()
    .on(context)
    .aggregate(np.mean)
)

In [ ]:
from trulens_eval import TruLlama
tru_query_engine_recorder = TruLlama(query_engine,
    app_id='LlamaIndex_DemoApp',
    feedbacks=[f_groundedness, f_qa_relevance, f_qs_relevance])

In [ ]:
with tru_query_engine_recorder as recording:
    query_engine.query('what is the maximum BSON document size in MongoDB')

In [ ]:
# The record of the app invocation can be retrieved from the `recording`:

rec = recording.get() # use .get if only one record
# recs = recording.records # use .records if multiple

display(rec)

In [ ]:
# tru.run_dashboard()

In [ ]:
# The results of the feedback functions can be rertireved from
# `Record.feedback_results` or using the `wait_for_feedback_result` method. The
# results if retrieved directly are `Future` instances (see
# `concurrent.futures`). You can use `as_completed` to wait until they have
# finished evaluating or use the utility method:

for feedback, feedback_result in rec.wait_for_feedback_results().items():
    print(feedback.name, feedback_result.result)

# See more about wait_for_feedback_results:
# help(rec.wait_for_feedback_results)

In [ ]:
records, feedback = tru.get_records_and_feedback(app_ids=["LlamaIndex_DemoApp"])

records.head()

In [ ]:
tru.get_leaderboard(app_ids=["LlamaIndex_DemoApp"])

In [ ]:
tru.run_dashboard()